In [14]:
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np
#import scipy.spatial.distance as dist
import mediapipe as mp
import cv2
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [26]:
def dtp(dist_mat):
    N, M = dist_mat.shape
    cost_mat = np.zeros((N + 1, M + 1), dtype = float)
    for i in range(1,N+1):
        cost_mat[i, 0]  = np.inf
    for j in range(1, M+1):
        cost_mat[0, j] = np.inf
    traceback_mat = np.zeros((N,M))
    for i in range(N):
        for j in range(M):
            min_list = [cost_mat[i, j], # match = 0
                        cost_mat[i, j+1],   #insert = 1
                        cost_mat[i+1, j]]   # deletion = 2
            index_min = np.argmin(min_list)
            cost_mat[i+1,j+1] = dist_mat[i, j] + min_list[index_min]
            traceback_mat[i,j] = index_min 
    i = N-1
    j = M -1
    path = [(i,j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i,j]
        if tb_type == 0: # đi chéo
            i = i-1
            j = j-1
        elif tb_type == 1: # đi xuống
            i = i - 1
        elif tb_type == 2: # đi ngang
            j = j - 1
        path.append((i,j))
    cost_mat = cost_mat[1:, 1:]
    return path[::-1], cost_mat


In [16]:
def distance_matrix(mat1, mat2):
    N = mat1.shape[1]
    M = mat2.shape[1]
    dist_mat = np.zeros((N,M))
    for i in range(N):
        for j in range(M):
            dist_mat[i, j] = np.linalg.norm(mat1[:, i] - mat2[:,j] )
    return dist_mat
    

In [17]:
def calculate_angle(a,b,c):
    a, b, c = np.array(a), np.array(b),  np.array(c)

    radians = np.arctan2(c[1]- b[1], c[0]-b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180/np.pi)
    if angle > 180:
        angle = 360 - 180
    return angle

In [18]:
def extract_vid(vid_path):
    cap = cv2.VideoCapture(vid_path)
    N = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    matrix = []
    count = 0
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
        while(cap.isOpened()):
            try:
                ret, frame = cap.read()
                count += 1
            #Change from BGR to RGB
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
                img.flags.writeable = False

                #Make detection
                result = pose.process(img)

                # Back the original color channels
                img.flags.writeable = True
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

                #Extract landmarks 
                try:
                    landmarks = result.pose_landmarks.landmark
                    #print(landmarks)
                except:
                    pass
                # Landmarks: là 1 list chứa tọa độ 33 điểm trên cơ thể
                theta = []
                for i in range(33):
                    for j in range(i+1, 33):
                        for k in range(j+1,33): 
                            a = [landmarks[i].x, landmarks[i].y]
                            b = [landmarks[j].x, landmarks[j].y]
                            c = [landmarks[k].x, landmarks[k].y]
                            angle = calculate_angle(a, b, c)
                            theta.append(angle)
                matrix.append(theta)

            except:
                pass
            if count == 150:
                break
        
        return np.array(matrix), N



In [19]:
mat1,n = extract_vid("Sources/viddtw1.mp4")
mat1.shape

(150, 5456)

In [20]:
mat2, m= extract_vid("Sources/viddtw2.mp4")
mat2.shape

(150, 5456)

In [21]:
dist_mat = distance_matrix(mat1, mat2)
dist_mat

array([[ 497.63648596,  481.29947438, 1024.65392984, ..., 1613.23933216,
        1557.75025969, 1822.7992832 ],
       [ 489.13609545,  472.62771814, 1026.22351716, ..., 1618.72716085,
        1554.27855737, 1833.38553658],
       [1172.85114991, 1185.20625892,  858.97348779, ...,  936.6918067 ,
        1110.38502683, 1062.93838389],
       ...,
       [ 730.22521755,  738.03759495,  830.59418031, ..., 1257.20091763,
        1249.39521398, 1437.97176941],
       [ 491.61483373,  482.22961073,  921.24758362, ..., 1493.70723729,
        1471.22437344, 1668.82317444],
       [1380.47699808, 1401.00532928,  988.10773405, ...,  638.05603374,
         932.23157731,  743.17058232]])

In [22]:
dist_mat.shape

(5456, 5456)

In [23]:
dist_mat[0,1]

481.2994743782194

In [27]:
path, cost_mt= dtp(dist_mat)
path

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (7, 8),
 (8, 9),
 (9, 10),
 (10, 10),
 (11, 10),
 (12, 11),
 (13, 11),
 (14, 12),
 (15, 13),
 (16, 14),
 (17, 15),
 (18, 16),
 (19, 17),
 (20, 18),
 (21, 19),
 (22, 20),
 (23, 21),
 (24, 22),
 (25, 23),
 (26, 24),
 (27, 25),
 (27, 26),
 (28, 27),
 (29, 28),
 (30, 29),
 (30, 30),
 (31, 31),
 (32, 32),
 (33, 33),
 (34, 34),
 (35, 35),
 (36, 36),
 (37, 37),
 (37, 38),
 (38, 39),
 (39, 40),
 (40, 40),
 (41, 40),
 (42, 41),
 (43, 41),
 (44, 42),
 (45, 43),
 (46, 44),
 (47, 45),
 (48, 46),
 (49, 47),
 (50, 48),
 (51, 49),
 (52, 50),
 (53, 51),
 (54, 52),
 (55, 53),
 (56, 54),
 (57, 55),
 (58, 56),
 (59, 57),
 (59, 58),
 (59, 59),
 (60, 60),
 (61, 61),
 (62, 62),
 (63, 63),
 (64, 64),
 (65, 65),
 (66, 66),
 (66, 67),
 (67, 68),
 (68, 69),
 (69, 69),
 (70, 69),
 (71, 70),
 (72, 70),
 (73, 71),
 (74, 72),
 (75, 73),
 (76, 74),
 (77, 75),
 (78, 76),
 (79, 77),
 (80, 78),
 (81, 79),
 (82, 80),
 (83, 81),
 (84, 82),
 (85, 83)